In [1]:
# Import libraries
import xgboost as xgb
import pandas as pd
import numpy as np

import sklearn.datasets
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [2]:
# Define global variables
SEED = 123

In [3]:
# Read data
churn_data = pd.read_csv("churn_data.csv", index_col='Id')
print(churn_data.head())

    avg_dist  avg_rating_by_driver  avg_rating_of_driver  avg_inc_price  \
Id                                                                        
0       3.67                   5.0                   4.7           1.10   
1       8.26                   5.0                   5.0           1.00   
2       0.77                   5.0                   4.3           1.00   
3       2.36                   4.9                   4.6           1.14   
4       3.13                   4.9                   4.4           1.19   

    inc_pct  weekday_pct  fancy_car_user  city_Carthag  city_Harko  \
Id                                                                   
0      15.4         46.2            True             0           1   
1       0.0         50.0           False             1           0   
2       0.0        100.0           False             1           0   
3      20.0         80.0            True             0           1   
4      11.8         82.4           False             0

In [4]:
churn_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50000 entries, 0 to 49999
Data columns (total 13 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   avg_dist                     50000 non-null  float64
 1   avg_rating_by_driver         49799 non-null  float64
 2   avg_rating_of_driver         41878 non-null  float64
 3   avg_inc_price                50000 non-null  float64
 4   inc_pct                      50000 non-null  float64
 5   weekday_pct                  50000 non-null  float64
 6   fancy_car_user               50000 non-null  bool   
 7   city_Carthag                 50000 non-null  int64  
 8   city_Harko                   50000 non-null  int64  
 9   phone_iPhone                 50000 non-null  int64  
 10  first_month_cat_more_1_trip  50000 non-null  int64  
 11  first_month_cat_no_trips     50000 non-null  int64  
 12  month_5_still_here           50000 non-null  int64  
dtypes: bool(1), floa

In [5]:
cancer = sklearn.datasets.load_breast_cancer()
print(type(cancer))
print(cancer.keys())

<class 'sklearn.utils.Bunch'>
dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename'])


# 1. Classification with XGBoost

This chapter will introduce you to the fundamental idea behind XGBoost—boosted learners. Once you understand how XGBoost works, you'll apply it to solve a common classification problem found in industry: predicting whether a customer will stop being a customer at some point in the future.

## 1.1 Welcome to the course!

1. Welcome to the course!
>Hi, my name is Sergey Fogelson and I'm the instructor for Datacamp's course on Gradient Boosted Trees With XGBoost. I'm a data scientist working in the media industry and have used XGBoost extensively on a variety of machine learning problems. I've created this course with DataCamp to help others quickly understand how to use this very popular implementation of gradient boosting. Let's get started.

2. Before we get to XGBoost...
>In order to understand XGBoost, we need to have some handle on the broader topics of supervised classification, decision trees, and boosting, which we will cover throughout this chapter. To begin, let's briefly review what

3. Supervised learning
>supervised learning is and the kinds of problems its methods can be applied to. At its core, supervised learning, which is the kind of learning problems that XGBoost can be applied to, relies on labeled data. That is, you have some understanding of the past behavior of the problem you're trying to solve or what you're trying to predict.

4. Supervised learning example
>For example, assessing whether a specific image contains a person's face, is a classification problem. Here the training data are images converted into vectors of pixel values, and the labels are either 1 when the image contains a face or 0 when the image doesn't contain a face. Given this, there are two kinds of supervised learning problems that account for the vast majority of use-cases: classification problems and regression problems. We will only talk about classification problems here and leave regression to chapter 2.

5. Supervised learning: Classification
>Classification problems involve predicting either binary or multi-class outcomes.

6. Binary classification example
>For example, predicting whether a person will purchase an insurance package given some quote is a binary supervised learning problem,

7. Multi-class classification example
>and predicting whether a picture contains one of several species of birds is a multi-class supervised learning problem. When dealing with binary supervised learning problems,

8. AUC: Metric for binary classification models
>the AUC, or Area Under the Receiver Operating Characteristic Curve, is the most versatile and common evaluation metric used to judge the quality of a binary classification model. It is simply the probability that a randomly chosen positive data point will have a higher rank than a randomly chosen negative data point for your learning problem. So, a higher AUC means a more sensitive, better performing model. When dealing with multi-class classification problems,

9. Accuracy score and confusion matrix
>it is common to use the accuracy score (higher is better) and to look at the overall confusion matrix to evaluate the quality of a model.

10. Review
>Some common algorithms for classification problems include logistic regression and decision trees. If you want a deeper review, check out DataCamp's introductory course on supervised learning.

11. Other supervised learning considerations
>All supervised learning problems, including classification problems, require that the data is structured as a table of feature vectors, where the features themselves (also called attributes or predictors) are either numeric or categorical. Furthermore, it is usually the case that numeric features are scaled to aid in either feature interpretation or to ensure that the model can be trained properly (for example, numerical feature scaling is essential to ensure properly trained support vector machine models). Categorical features are also almost always encoded before applying supervised learning algorithms, most commonly using one-hot encoding. Finally, other kinds of supervised learning problems exist, so I'll mention them here briefly.

12. Ranking
>Ranking problems involve predicting an ordering on a set of choices (like google search suggestions),

13. Recommendation
>and recommendation problems involve recommending an item or set of items to a user based on his/her consumption history and profile (like Netflix).

14. Let's practice!
>Now that you've been reminded about the basics of classification problems, let's get to work!

### Remarkable:

- **Best metrics**:
    * **Binary supervised learning problems**:
        + **AUC**, or Area Under the Receiver Operating Characteristic Curve. Higher AUC means more sensitive and better performing model.<br>
        + **Confusion Matrix** to evaluate the quality of a model.

- Supervised learning **considerations**:
    * Features can be either numeric or categorical
    * Numeric features should be scaled (Z-scored)
    * Categorical features should be encoded (one-hot)

## 1.2 Which of these is a classification problem?

**Instructions**

Given below are 4 potential machine learning problems you might encounter in the wild. Pick the one that is a classification problem.

**Possible Answers**

1. Given past performance of stocks and various other financial data, predicting the exact price of a given stock (Google) tomorrow.
2. Given a large dataset of user behaviors on a website, generating an informative segmentation of the users based on their behaviors.
3. <font color=red>Predicting whether a given user will click on an ad given the ad content and metadata associated with the user.</font> **Correct!**
4. Given a user's past behavior on a video platform, presenting him/her with a series of recommended videos to watch next.

**Results**

<font color=darkgreen>Well done! This is indeed a classification problem.</font>

## 1.3 Which of these is a binary classification problem?

**Instructions**

Great! A classification problem involves predicting the category a given data point belongs to out of a finite set of possible categories. Depending on how many possible categories there are to predict, a classification problem can be either binary or multi-class. Let's do another quick refresher here. Your job is to pick the binary classification problem out of the following list of supervised learning problems.

**Possible Answers**

1. <font color=red>Predicting whether a given image contains a cat.</font> **Correct!**
2. Predicting the emotional valence of a sentence (Valence can be positive, negative, or neutral).
3. Recommending the most tax-efficient strategy for tax filing in an automated accounting system.
4. Given a list of symptoms, generating a rank-ordered list of most likely diseases.

**Results**

<font color=darkgreen>Correct! A binary classification problem involves picking between 2 choices.</font>

## 1.4 Introducing XGBoost

1. Introducing XGBoost
>Now let's talk about what you're actually here for, the hottest library in supervised machine learning, XGBoost.

2. What is XGBoost?
>XGBoost is an incredibly popular machine learning library for good reason. It was developed originally as a C++ command-line application. After winning a popular machine learning competition, the package started being adopted within the ML community. As a result, bindings, or functions that tapped into the core C++ code, started appearing in a variety of other languages, including Python, R, Scala, and Julia. We will cover the Python API in this course.

3. What makes XGBoost so popular?
>What makes XGBoost so popular? Its speed and performance. Because the core XGBoost algorithm is parallelizable, it can harness all of the processing power of modern multi-core computers. Furthermore, it is parallelizable onto GPU's and across networks of computers, making it feasible to train models on very large datasets on the order of hundreds of millions of training examples. However, XGBoost's speed isn't the package's real draw. Ultimately, a fast but poorly performing machine learning algorithm is not going to have wide adoption within the community. What makes XGBoost so popular is that it consistently outperforms almost all other single-algorithm methods in machine learning competitions and has been shown to achieve state-of-the-art performance on a variety of benchmark machine learning datasets. Here's an example of how we can use XGBoost using a classification problem.

4. Using XGBoost: a quick example
>In lines 1-4, we import the libraries or functions we will be using, including xgboost, and the train/test/split function from scikit-learn. Remember, you always build a machine learning model using train/test splits of your data, where some portion of your data is used for training, and the remainder is held-out for testing to ensure that your model doesn't overfit and can generalize to unseen data. In lines 5 and 6 we load our data in from file and split the entire dataset into a matrix of samples by features, called X by convention, and a vector of target values, called y by convention. In line 7 we create our train/test split, keeping 20% of the data for testing. In line 8 we instantiate our xgboost classifier instance with some parameters that we will cover shortly. Lines 9 and 10 should appear familiar to you. XGBoost has a scikit-learn compatible api, and this is it! It uses the fit/predict pattern that you should have seen before, where we fit, or train, our algorithm on the training set, and then evaluate it by generating predictions using the test set and comparing our predictions to the actual target labels on the test set. Lines 11 and 12 evaluate the accuracy of the trained model on the test set and print the results to screen.

5. Let's begin using XGBoost!
>Given that XGBoost is this popular, let's get to using it already!

In [6]:
X, y = churn_data.iloc[:,:-1], churn_data.iloc[:,-1]

X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=SEED)

xg_cl = xgb.XGBClassifier(objective='binary:logistic', use_label_encoder=False, n_estimators=10, seed=SEED, eval_metric='error')
xg_cl.fit(X_train, y_train)

preds = xg_cl.predict(X_test)
accuracy = float(np.sum(preds==y_test))/y_test.shape[0]

print("accuracy: %f" % (accuracy))

accuracy: 0.758200


## 1.5 XGBoost: Fit/Predict

It's time to create your first XGBoost model! As Sergey showed you in the video, you can use the scikit-learn .fit() / .predict() paradigm that you are already familiar to build your XGBoost models, as the xgboost library has a scikit-learn compatible API!

Here, you'll be working with churn data. This dataset contains imaginary data from a ride-sharing app with user behaviors over their first month of app usage in a set of imaginary cities as well as whether they used the service 5 months after sign-up. It has been pre-loaded for you into a DataFrame called churn_data - explore it in the Shell!

Your goal is to use the first month's worth of data to predict whether the app's users will remain users of the service at the 5 month mark. This is a typical setup for a churn prediction problem. To do this, you'll split the data into training and test sets, fit a small xgboost model on the training set, and evaluate its performance on the test set by computing its accuracy.

pandas and numpy have been imported as pd and np, and train_test_split has been imported from sklearn.model_selection. Additionally, the arrays for the features and the target have been created as X and y.

**Instructions**
1. Import xgboost as xgb.
2. Create training and test sets such that 20% of the data is used for testing. Use a random_state of 123.
3. Instantiate an XGBoostClassifier as xg_cl using xgb.XGBClassifier(). Specify n_estimators to be 10 estimators and an objective of 'binary:logistic'. Do not worry about what this means just yet, you will learn about these parameters later in this course.
4. Fit xg_cl to the training set (X_train, y_train) using the .fit() method.
5. Predict the labels of the test set (X_test) using the .predict() method and hit 'Submit Answer' to print the accuracy.

**Results**

<font color=darkgreen>Well done! Your model has an accuracy of around 74%. In Chapter 3, you'll learn about ways to fine tune your XGBoost models. For now, let's refresh our memories on how decision trees work. See you in the next video!</font>

In [7]:
# Create arrays for the features and the target: X, y
X, y = churn_data.iloc[:,:-1], churn_data.iloc[:,-1]

# Create the training and test sets
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size=0.2, random_state=SEED)

# Instantiate the XGBClassifier: xg_cl
xg_cl = xgb.XGBClassifier(use_label_encoder=False, eval_metric='error', n_estimators=10, objective='binary:logistic', seed=SEED)

# Fit the classifier to the training set
xg_cl.fit(X_train, y_train)

# Predict the labels of the test set: preds
preds = xg_cl.predict(X_test)

# Compute the accuracy: accuracy
accuracy = float(np.sum(preds==y_test))/y_test.shape[0]
print("Accuracy: %f" % (accuracy))

Accuracy: 0.758200


## 1.6 What is a decision tree?


1. What is a decision tree?
>Because XGBoost is usually used with trees as base learners, we need to understand what an individual decision tree is, and how it works.

2. Visualizing a decision tree
>Here is an example decision tree. As you can see, it has a single question that is being asked at each decision node, and only 2 possible choices, at the very bottom of each decision tree, there is a single possible decision. In this example decision tree for whether to purchase a vehicle, the first question you ask is whether it has been road-tested. If it hasn't, you immediately decide not to buy, otherwise, you continue asking questions, such as what the vehicle's mileage is, and, if its age is old or recent. At bottom, every possible decision will eventually lead to a choice, some taking many fewer questions to get to those choices than others.
<br><br>https://www.ibm.com/support/knowledgecenter/en/SS3RA7_15.0.0/com.ibm.spss.modeler.help/nodes_treebuilding.htm

3. Decision trees as base learners
>The concept of a base learner will be covered more extensively later, but for now, just think of any individual learning algorithm in an ensemble algorithm as a base learner. This is important because XGBoost itself is an ensemble learning method in that it uses the outputs of many models for a final prediction. Anyway, as you saw in the previous slide, a decision tree is a learning method that involves a tree-like graph to model either a continuous or categorical choice given some data. It is composed of a series of binary decisions (yes/no or true/false questions) that when answered in succession ultimately yield a prediction about the data at hand (these predictions happen at the leaves of the tree).

4. Decision trees and CART
>Decision trees are constructed iteratively (that is, one binary decision at a time) until some stopping criterion is met (the depth of the tree reaches some pre-defined maximum value, for example). During construction, the tree is built one split at a time, and the way that a split is selected (that is, what feature to split on and where in the feature's range of values to split) can vary, but involves choosing a split point that segregates the target values better (puts each target category into buckets that are increasingly dominated by just one category) until all (or nearly all) values within a given split are exclusively of one category or another. Using this process, each leaf of the decision tree will have a single category in the majority, or should be exclusively of one category.

5. Individual decision trees tend to overfit
>Individual decision trees in general are low-bias, high-variance learning models.
<br><br>http://scott.fortmann-roe.com/docs/BiasVariance.html

6. Individual decision trees tend to overfit
>That is, they are very good at learning relationships within any data you train them on, but they tend to overfit the data you use to train them on and usually generalize to new data poorly. XGBoost uses a slightly different kind of a decision tree,
<br><br>http://scott.fortmann-roe.com/docs/BiasVariance.html

7. CART: Classification and Regression Trees
>called a classification and regression tree, or CART. Whereas for the decision trees described above the leaf nodes always contain decision values, CART trees contain a real-valued score in each leaf, regardless of whether they are used for classification or regression. The real-valued scores can then be thresholded to convert into categories for classification problems if necessary.

8. Let's work with some decision trees!
>Awesome, let's get to working with some decision trees!

## 1.7 Decision trees

Your task in this exercise is to make a simple decision tree using scikit-learn's DecisionTreeClassifier on the breast cancer dataset that comes pre-loaded with scikit-learn.

This dataset contains numeric measurements of various dimensions of individual tumors (such as perimeter and texture) from breast biopsies and a single outcome value (the tumor is either malignant, or benign).

We've preloaded the dataset of samples (measurements) into X and the target values per tumor into y. Now, you have to split the complete dataset into training and testing sets, and then train a DecisionTreeClassifier. You'll specify a parameter called max_depth. Many other parameters can be modified within this model, and you can check all of them out here.

**Instructions**
1. Import:
    - train_test_split from sklearn.model_selection.
    - DecisionTreeClassifier from sklearn.tree.
2. Create training and test sets such that 20% of the data is used for testing. Use a random_state of 123.
3. Instantiate a DecisionTreeClassifier called dt_clf_4 with a max_depth of 4 (maximum depth of the tree). This parameter specifies the maximum number of successive split points you can have before reaching a leaf node.
4. Fit the classifier to the training set and predict the labels of the test set.

**Results**

<font color=darkgreen>Great work! It's now time to learn about what gives XGBoost its state-of-the-art performance: Boosting.</font>

In [8]:
# Create arrays for the features and the target: X, y
X, y = cancer.data, cancer.target

# Create the training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

# Instantiate the classifier: dt_clf_4
dt_clf_4 = DecisionTreeClassifier(max_depth=4)

# Fit the classifier to the training set
dt_clf_4.fit(X_train, y_train)

# Predict the labels of the test set: y_pred_4
y_pred_4 = dt_clf_4.predict(X_test)

# Compute the accuracy of the predictions: accuracy
accuracy = float(np.sum(y_pred_4==y_test))/y_test.shape[0]
print("accuracy:", accuracy)

accuracy: 0.9736842105263158


## 1.8 What is Boosting?

1. What is Boosting?
>Now that we've reviewed both supervised learning and the basics of decision trees, lets talk about the core concept that gives XGBoost its state-of-the-art performance, boosting.

2. Boosting overview
>At bottom, boosting isn't really a specific machine learning algorithm, but a concept that can be applied to a set of machine learning models. So, its really a meta-algorithm. Specifically, it is an ensemble meta-algorithm primarily used to reduce any given single learner's variance and to convert many weak learners into an arbitrarily strong learner.

3. Weak learners and strong learners
>A weak learner is any machine learning algorithm that is just slightly better than chance. So, a decision tree that can predict some outcome slightly more frequently than pure randomness would be considered a weak learner. The principal insight that allows XGBoost to work is the fact that you can use boosting to convert a collection of weak learners into a strong learner. Where a strong learner is any algorithm that can be tuned to achieve arbitrarily good performance for some supervised learning problem.

4. How boosting is accomplished
>How is this accomplished? By iteratively learning a set of weak models on subsets of the data you have at hand, and weighting each of their predictions according to each weak learner's performance. You then combine all of the weak learners' predictions multiplied by their weights to obtain a single final weighted prediction that is much better than any of the individual predictions themselves. It's kind of incredible that this works as well as it does.

5. Boosting example
>Here is a very basic example of boosting using 2 decision trees. This example comes from the XGBoost documentation and shows that given a specific example, each tree gives a different prediction score depending on the data it sees. The prediction scores for each possibility are summed across trees and the prediction is simply the sum of the scores across both trees. Here, you can see that whatever it was we were trying to predict, the little boy had a higher predicted score summed across both trees than the old man.
<br><br>https://xgboost.readthedocs.io/en/latest/model.html

6. Model evaluation through cross-validation
>Since we will be working with XGBoost's learning API for model evaluation next, it's a good idea to briefly provide you with an example that shows how model evaluation using cross-validation works with XGBoost's learning API (which is different from the scikit-learn compatible API) as it has cross-validation capabilities baked in. As a refresher, cross-validation is a robust method for estimating the expected performance of a machine learning model on unseen data by generating many non-overlapping train/test splits on your training data and reporting the average test set performance across all data splits.

7. Cross-validation in XGBoost example
>So, in lines 1 and 2 we import what we will be using. In line 3, we load in our example dataset. In line 4, we convert our dataset into an optimized data structure that the creators of XGBoost made that gives the package its lauded performance and efficiency gains called a DMatrix. In the previous exercise, the input datasets were converted into DMatrix data on the fly, but when we use the XGBoost cv object, which is part of XGBoost's learning api we have to first explicitly convert our data into a DMatrix. So, that's what we are doing here before we run our cross-validation. In line 5, we are creating a parameter dictionary to pass into our cross-validation. This is necessary because the cv method has no idea what kind of XGBoost model we are using and expects us to provide that information as a dictionary of appropriate key-value pairs. Our parameter dictionary here is bare-bones, only providing the objective function we would like to use and the maximum depth that every tree can grow to. In line 6, we call the cv method and pass in our DMatrix object storing all of our data, the parameter dictionary, the number of cross-validation folds, how many trees we want to build, what metric we want to compute, and whether we want our output to be stored as a pandas dataframe. In line 7 we simply convert our metrics into an accuracy and output the results to screen.

8. Let's practice!
>Now it's your turn!

In [9]:
#Cross-validation in XGBoost example

churn_dmatrix = xgb.DMatrix(data  = churn_data.iloc[:,:-1],
                            label = churn_data.month_5_still_here)

params = {"objective"        : "binary:logistic", 
          "max_depth"        : 4}

cv_results = xgb.cv(dtrain          = churn_dmatrix, 
                    params          = params, 
                    nfold           = 4,
                    num_boost_round = 10, 
                    metrics         = "error", 
                    as_pandas       = True,
                    seed            = SEED)
print(cv_results)
print("\nAccuracy: %f" %((1-cv_results["test-error-mean"]).iloc[-1]))
# Previous Accuracy: 0.758200

   train-error-mean  train-error-std  test-error-mean  test-error-std
0          0.269240         0.002533          0.27090        0.002357
1          0.259700         0.002649          0.26052        0.005451
2          0.256413         0.001775          0.25774        0.003399
3          0.249673         0.000833          0.25100        0.003684
4          0.246146         0.001230          0.24678        0.004801
5          0.243687         0.001459          0.24508        0.003138
6          0.242180         0.001519          0.24314        0.003488
7          0.241374         0.001202          0.24156        0.003787
8          0.240173         0.001107          0.24128        0.004357
9          0.239020         0.001076          0.24082        0.003654

Accuracy: 0.759180


## 1.9 Measuring accuracy

You'll now practice using XGBoost's learning API through its baked in cross-validation capabilities. As Sergey discussed in the previous video, XGBoost gets its lauded performance and efficiency gains by utilizing its own optimized data structure for datasets called a DMatrix.

In the previous exercise, the input datasets were converted into DMatrix data on the fly, but when you use the xgboost cv object, you have to first explicitly convert your data into a DMatrix. So, that's what you will do here before running cross-validation on churn_data.

**Instructions**

1. Create a DMatrix called churn_dmatrix from churn_data using xgb.DMatrix(). The features are available in X and the labels in y.
2. Perform 3-fold cross-validation by calling xgb.cv(). dtrain is your churn_dmatrix, params is your parameter dictionary, nfold is the number of cross-validation folds (3), num_boost_round is the number of trees we want to build (5), metrics is the metric you want to compute (this will be "error", which we will convert to an accuracy).

**Results**

<font color=darkgreen>Nice work. cv_results stores the training and test mean and standard deviation of the error per boosting round (tree built) as a DataFrame. From cv_results, the final round 'test-error-mean' is extracted and converted into an accuracy, where accuracy is 1-error. The final accuracy of around 75% is an improvement from earlier!</font>

In [10]:
# Create arrays for the features and the target: X, y
X, y = churn_data.iloc[:,:-1], churn_data.iloc[:,-1]

# Create the DMatrix from X and y: churn_dmatrix
churn_dmatrix = xgb.DMatrix(data=X, label=y)

# Create the parameter dictionary: params
params = {"objective":"reg:logistic", "max_depth":4}

# Perform cross-validation: cv_results
cv_results = xgb.cv(dtrain=churn_dmatrix, params=params, 
                  nfold=4, num_boost_round=10, 
                  metrics="error", as_pandas=True, seed=SEED)

# Print cv_results
print(cv_results)

# Print the accuracy
print("\nAccuracy: %f" %((1-cv_results["test-error-mean"]).iloc[-1]))
# Previous Accuracy: 0.758200

   train-error-mean  train-error-std  test-error-mean  test-error-std
0          0.269240         0.002533          0.27090        0.002357
1          0.259700         0.002649          0.26052        0.005451
2          0.256413         0.001775          0.25774        0.003399
3          0.249673         0.000833          0.25100        0.003684
4          0.246146         0.001230          0.24678        0.004801
5          0.243687         0.001459          0.24508        0.003138
6          0.242180         0.001519          0.24314        0.003488
7          0.241374         0.001202          0.24156        0.003787
8          0.240173         0.001107          0.24128        0.004357
9          0.239020         0.001076          0.24082        0.003654

Accuracy: 0.759180


## 1.10 Measuring AUC

Now that you've used cross-validation to compute average out-of-sample accuracy (after converting from an error), it's very easy to compute any other metric you might be interested in. All you have to do is pass it (or a list of metrics) in as an argument to the metrics parameter of xgb.cv().

Your job in this exercise is to compute another common metric used in binary classification - the area under the curve ("auc"). As before, churn_data is available in your workspace, along with the DMatrix churn_dmatrix and parameter dictionary params.

**Instructions**

1. Perform 3-fold cross-validation with 5 boosting rounds and "auc" as your metric.
2. Print the "test-auc-mean" column of cv_results.

**Results**

<font color=darkgreen>Fantastic! An AUC of 0.84 is quite strong. As you have seen, XGBoost's learning API makes it very easy to compute any metric you may be interested in. In Chapter 3, you'll learn about techniques to fine-tune your XGBoost models to improve their performance even further. For now, it's time to learn a little about exactly when to use XGBoost.</font>

In [11]:
# Perform cross_validation: cv_results
cv_results = xgb.cv(dtrain=churn_dmatrix, params=params, 
                  nfold=4, num_boost_round=10, 
                  metrics="auc", as_pandas=True, seed=SEED)

# Print cv_results
print(cv_results)

# Print the AUC
print("\nAUC:", (cv_results["test-auc-mean"]).iloc[-1])

   train-auc-mean  train-auc-std  test-auc-mean  test-auc-std
0        0.789012       0.000578       0.787374      0.002438
1        0.809241       0.004920       0.807963      0.006237
2        0.817544       0.002459       0.815383      0.001718
3        0.829441       0.001968       0.826812      0.001211
4        0.832774       0.001859       0.829991      0.001673
5        0.836025       0.001083       0.833360      0.002424
6        0.838328       0.000790       0.835769      0.002500
7        0.839759       0.000767       0.836970      0.002875
8        0.840844       0.000789       0.837638      0.002979
9        0.842057       0.000617       0.838576      0.003167

AUC: 0.8385765


## 1.11 When should I use XGBoost?

1. When should I use XGBoost?
>So, given everything we've said about XGBoost, when should (and shouldn't) you use it?

2. When to use XGBoost
>Given that I've already talked a bit about when and where XGBoost shines, some of this shouldn't come as a surprise to you. You should consider using XGBoost for any supervised machine learning task that fits the following criteria: You have a large number of training examples. Although your definition of large can vary, I intend it to mean a dataset that has few features and at least 1000 examples. However, in general, as long as the number of features in your training set is smaller than the number of examples you have, you should be fine. Finally, XGBoost tends to do well when you have a mixture of categorical and numeric features, or when you have just numeric features.

3. When to NOT use XGBoost
>When should you not use XGBoost? The most important kinds of problems where XGBoost is a suboptimal choice involve either those that have found success using other state-of-the-art algorithms or those that suffer from dataset size issues. Specifically, XGBoost is not ideally suited for image recognition, computer vision, or natural language processing and understanding problems, as those kinds of problems can be much better tackled using deep learning approaches. In terms of dataset size problems, XGBoost is not suitable when you have very small training sets ( less than 100 training examples) or when the number of training examples is significantly smaller than the number of features being used for training.

4. Let's practice!
>Ok, let's finish off what you learned in chapter 1 with one last multiple choice question!

## 1.12 Using XGBoost

XGBoost is a powerful library that scales very well to many samples and works for a variety of supervised learning problems. That said, as Sergey described in the video, you shouldn't always pick it as your default machine learning library when starting a new project, since there are some situations in which it is not the best option. In this exercise, your job is to consider the below examples and select the one which would be the best use of XGBoost.

**Possible Answers**

1. Visualizing the similarity between stocks by comparing the time series of their historical prices relative to each other.
2. Predicting whether a person will develop cancer using genetic data with millions of genes, 23 examples of genomes of people that didn't develop cancer, 3 genomes of people who wound up getting cancer.
3. Clustering documents into topics based on the terms used in them.
4. <font color=red>Predicting the likelihood that a given user will click an ad from a very large clickstream log with millions of users and their web interactions</font>. **Correct!**.

**Results**

<font color=darkgreen>Correct! Way to end the chapter. Time to apply XGBoost to solve regression problems!</font>

# Aditional material

- Datacamp course: https://learn.datacamp.com/courses/extreme-gradient-boosting-with-xgboost
- Xgboost documentation: https://xgboost.readthedocs.io/en/latest/
- sklearn.tree.DecisionTreeClassifier documentation: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier